# Webscraping Netnaija 
### Author: Sachimugu 
Email: <a href="mailto:sachimugu@outlook.com"> Email</a>

### Importing all relevant libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Creating resuable fuctions

In [2]:
#function to load a url and parse it content
def browse(url): #browse function
    page= requests.get(url) #to load a url
    soup= BeautifulSoup(page.content, 'lxml') # parse it content
    return soup


#function to get the next page link
def nextpage(soup): #nextpage function
    try:
        nxtpage=soup.find('a', class_="next page-numbers", href=True)['href']
        return nxtpage #return the page link
    except:
        return #return nothing if at the last page
        

###  Loop to get link of all pages

In [3]:
#Base url
url="https://www.thenetnaija.com/videos/movies" #open on your browser to get a feel of the site architecture

page_link=[] # empty list to store list of all pages

while True:
    soup=browse(url) # load the base url
    url=nextpage(soup)# get next page on base url append it to page list and become the new base url
    if not url:
        break
    page_link.append(url)
    

In [4]:
len(page_link)

152

### Loop to get the links to all movies on each page

In [5]:
movie_links=[] # empty list to store movie links form each page
for page in page_link: # a for loop to get each page from the page list
    soup= browse(page) # load each page and parse
    
    # this series of code get all link to movies on each page and append it to movie_links
    video_files=soup.find("div", class_="video-files")
    class_info=video_files.findAll("div", class_="info")
    for x in class_info:
        link=x.find("a", href=True)['href']
        movie_links.append(link)
        

### Empty list to store data we need about each movie

In [7]:
titles=[]  # movie titles
movie_linkss=[] # movie links
movie_types=[] #video type
time_of_uplos=[] # date of upload
movie_lengths=[] # lenght of movie
num_of_comments=[] #numbers of comment
mo_summarys=[] # moive summary
Genres=[] #movie genre
Release_Dates=[] #release date
Starss=[] # actors and actress
Languages=[] #movie language
Subtitles=[] #available subtitle
imdb_links=[] #imdb link

### Getting data about each movie

In [8]:

for link in movie_links:
    soup= browse(link) # browse movie link and parse
    
    #This series of code get the requried data and append to the data list
    try:
        title=soup.find('h1', class_="page-h1").text 
        titles.append(title)
    except:
        titles.append(' ')
    post_meta=soup.find("div", class_="post-meta")
    try:
        movie_link=post_meta.find('a', href=True)['href']
        movie_linkss.append(movie_link)
    except:
        movie_links.append(' ')
    meta_one=soup.findAll('span', class_='meta-one')
    try:
        movie_type=meta_one[0].text.split()
        movie_types.append(movie_type)
    except:
        movie_types.append(' ')
    x=meta_one[1].text.split()
    try:
        time_of_uplo=' '.join(x)
        time_of_uplos.append(time_of_uplo)
    except:
        time_of_uplos.append(' ')
    try:
        movie_length=meta_one[2].text.split()
        movie_lengths.append(movie_length)
    except:
        movie_lengths.append(' ')
    try:
        num_of_comment=meta_one[3].text.split()
        num_of_comments.append(num_of_comment)
    except:
        num_of_comments.append('0')
    try:
        mo_summary=soup.find('p').next_element
        mo_summarys.append(mo_summary)
    except:
        mo_summarys.append(' ')
    try:
        block=soup.find('blockquote', class_='quote-content')
        y=block.findAll('p')
        
        try:
            Genre=y[1].text.split(':')[1:]
            Genres.append(Genre)
        except:
            Genres.append('missing')
        try:
            Release_Date=y[2].text
            Release_Dates.append(Release_Date)
        except:
            Release_Dates.append('missing')

        try:
            Stars=y[3].text.split(':')[1:]
            Starss.append(Stars)
        except:
            Starss.append('missing')

        try:
            Language=y[5].text.split(':')[1:]
            Languages.append(Language)
        except:
              Languages.append('missing')
        try:
            Subtitle=y[6].text.split(':')[1:]
            Subtitles.append(Subtitle)
        except:
            Subtitles.append('missing')

    except:
        Genres.append('missing')
        Release_Dates.append('missing')
        Starss.append('missing')
        Languages.append('missing')
        Subtitles.append('missing')
            
   
    try:
        imdb_link=block.find('a', href=True)['href']
        imdb_links.append(imdb_link)
    except:
         imdb_links.append('missing')
        

### Creating a table of all data with pandas dataframe

In [9]:
df=pd.DataFrame({"titles":titles,       
              "movie_types":movie_types,
              "time_of_uplos":time_of_uplos,
              "movie_lengths":movie_lengths,
              "num_of_comments":num_of_comments,
              "Genres":Genres,
              "Release_Dates":Release_Dates,
              "Starss":Starss,
              "Languages":Languages,
              "Subtitles":Subtitles,
              "movie_linkss":movie_linkss,  
              "imdb_links":imdb_links,
              "mo_summarys":mo_summarys,   
             })

### save data in cvs and excel format 

In [11]:
df.to_csv('./Data/netnaija_movie.csv')
df.to_excel('./Data/netnaija_movie.xlsx')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2731 entries, 0 to 2730
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titles           2731 non-null   object
 1   movie_types      2731 non-null   object
 2   time_of_uplos    2731 non-null   object
 3   movie_lengths    2731 non-null   object
 4   num_of_comments  2731 non-null   object
 5   Genres           2731 non-null   object
 6   Release_Dates    2731 non-null   object
 7   Starss           2731 non-null   object
 8   Languages        2731 non-null   object
 9   Subtitles        2731 non-null   object
 10  movie_linkss     2731 non-null   object
 11  imdb_links       2731 non-null   object
 12  mo_summarys      2731 non-null   object
dtypes: object(13)
memory usage: 277.5+ KB


In [13]:
df.head(5)

,titles,movie_types,time_of_uplos,movie_lengths,num_of_comments,Genres,Release_Dates,Starss,Languages,Subtitles,movie_linkss,imdb_links,mo_summarys
0,Sweet Carolina (2021),[Movies],Sep 07,[01:24:40],[25],"[ Comedy, Family, Romance]","Release Date: May 15, 2021","[ Lacey Chabert, Tyler Hynes, Gregory Harrison]",[ English],[ English],https://www.thenetnaija.com/videos/movies,https://www.imdb.com/title/tt14477372/,[A family's loss becomes an unexpected gift]
1,Grace & Saleem (2019),[Movies],Sep 06,[01:24:01],[15],"[ Comedy, Drama, Romance]","Release Date: Aug 19, 2019","[ Dru Castiglione, Kyle Daniel Hernandez, Shev...",[ English],[ English],https://www.thenetnaija.com/videos/movies,https://www.imdb.com/title/tt10649220/,"An unintended road trip, a comical first dinne..."
2,"Barbie: Big City, Big Dreams (2021)",[Movies],Sep 06,[01:03:00],[37],"[ Animation, Family, Musical]","Release Date: Aug 20, 2021","[ America Young, Amber Eliese May, Greg Chun]",[ English],[ English],https://www.thenetnaija.com/videos/movies,https://www.imdb.com/title/tt15053428/,Barbie swaps the sunny shores of Malibu for th...
3,The J Team (2021),[Movies],Sep 04,[01:24:53],[38],"[ Comedy, Family, Musical]","Release Date: Sep 3, 2021","[ JoJo Siwa, Tisha Campbell, Laura Soltis, Kia...",[ English],[ English],https://www.thenetnaija.com/videos/movies,https://www.imdb.com/title/tt14006530/,The story follows a young girl named JoJo whos...
4,Karen (2021),[Movies],Sep 03,[40],0,"[ Horro, Thriller]","Release Date: Sep 3, 2021","[ Taryn Manning, Cory Hardrict, Gregory Alan W...",[ English],[ English],https://www.thenetnaija.com/videos/movies,https://www.imdb.com/title/tt12636872/,[Not all neighbors are friendly.]
